In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.0'
os.environ['SPARK_VERSION']=spark_version

# innstall java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# install findspark using pip
!pip install -q findspark


# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,338 kB]
Get:13 http://arch

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-30 10:10:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.60MB/s    in 0.2s    

2022-11-30 10:10:52 (5.60 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Toys_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Toys_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|          5|            0|          0|   N|                Y|          Five Stars|        Excellent!!!| 2015-08-31|
|         US|   24769659|R36ED1U38IELG8|B00D7JFOPC|     952062646|56 Pieces of Wood...|            Toys|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RDIJS7QYB6XNR|          5|            0|          0|   N|                Y|
|R36ED1U38IELG8|          5|            0|          0|   N|                Y|
| R1UE3RPRGCOLD|          2|            1|          1|   N|                Y|
|R298788GS6I901|          5|            0|          0|   N|                Y|
|  RNX4EXOBBPN5|          1|            1|          1|   N|                Y|
|R3BPETL222LMIM|          5|            0|          0|   N|                Y|
|R3SORMPJZO3F2J|          3|            2|          2|   N|                Y|
|R2RDOJQ0WBZCF6|          5|            0|          0|   N|                Y|
|R2B8VBEPB4YEZ7|          5|            0|          0|   N|                Y|
|R1CB783I7B0U52|          1|            0|          1|   N|     

In [7]:
# Filter the data and create a new DataFrame to retrieve all the rows where the total_votes count is equal to or greater than 20
total_votes_df = vine_df.filter(vine_df["total_votes"]>20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2MDI1TP46VSYD|          3|            9|         21|   N|                Y|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|
|R2GHOWIWDVLN18|          1|           28|         36|   N|     

In [11]:
# Create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
# Load in an SQL function to use columns
from pyspark.sql.functions import col
percent_votes_df = total_votes_df.filter((col("helpful_votes")/col("total_votes")) >= 0.5)
percent_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|
|R2GHOWIWDVLN18|          1|           28|         36|   N|                Y|
|R30PJ9XUHFTY5D|          5|           42|         43|   N|     

In [27]:
# Get the total number of 5 star reviews
total_star_reviews = percent_votes_df.filter(col("star_rating")==5).count()
total_star_reviews

28453

In [12]:
# Filter the DataFrame and create a new DataFrame that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_vine_df = percent_votes_df.filter(col("vine") == 'Y')
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ED60RC69CCQ6|          5|           22|         23|   Y|                N|
|R2JM687C525WR9|          3|           33|         33|   Y|                N|
| RUB4AUWGHG16G|          4|           25|         27|   Y|                N|
|R1S3IDYJUA6V2G|          5|           28|         32|   Y|                N|
| RZFLN8FD5Y2M6|          4|           25|         28|   Y|                N|
|R1V8NDXD9SETO5|          3|          320|        352|   Y|                N|
|R2H4VEFWHP50M3|          5|           26|         36|   Y|                N|
|R3DP6Y0A8WXUOV|          5|           21|         21|   Y|                N|
| R49L85GZIKH4D|          3|           33|         36|   Y|                N|
|R34JSTH01HK1J8|          4|           75|         80|   Y|     

In [13]:
# Repeat Step, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
unpaid_vine_df = percent_votes_df.filter(col("vine") == 'N')
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| ROP6ITXO8K5V2|          5|           23|         27|   N|                Y|
|R3ND1LVU7AXCVF|          1|           21|         21|   N|                Y|
|R1QS8AOD6HX3ED|          4|           59|         81|   N|                N|
|R2LWX4TZ67FWPT|          4|           50|         50|   N|                N|
|R29IYHPYD14AGI|          3|           84|         84|   N|                Y|
| RRO8C1IVZMD6H|          5|           22|         22|   N|                N|
|R2GHOWIWDVLN18|          1|           28|         36|   N|                Y|
|R30PJ9XUHFTY5D|          5|           42|         43|   N|                N|
|R2F7D9N3SSQC20|          4|           32|         34|   N|                N|
| RN4VK9FV66YPS|          5|           29|         33|   N|     

In [18]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)
# For paid vine program
from pyspark.sql.functions import count

# Total number of reviews
paid_vine_df.agg(count("review_id")).show()

+----------------+
|count(review_id)|
+----------------+
|            1203|
+----------------+



In [23]:
# The number of 5-star reviews
paid_star_reviews = paid_vine_df.filter(col("star_rating")==5).count()
paid_star_reviews

410

In [24]:
# For the unpaid vine program
# Total number of reviews
unpaid_vine_df.agg(count("review_id")).show()

+----------------+
|count(review_id)|
+----------------+
|           58018|
+----------------+



In [25]:
# The number of 5-star reviews
unpaid_star_reviews = unpaid_vine_df.filter(col("star_rating")==5).count()
unpaid_star_reviews

28043

In [28]:
# The percentage of 5-star reviews for the two types of review (paid vs unpaid)
percent_paid_reviews = (paid_star_reviews/total_star_reviews) * 100
percent_paid_reviews

1.4409728323902578

In [29]:
percent_unpaid_reviews = (unpaid_star_reviews/total_star_reviews) * 100
percent_unpaid_reviews

98.55902716760974